In [51]:
from tensorflow.keras import layers, Sequential
import tensorflow_datasets as tfds
import tensorflow.keras
import numpy as np
import re
import os
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
# this is used to remove stage direction if we don't want them
def remove_stage_dir(text):
    text = re.sub("[\<].*?[\>]", "", text)
    text = re.sub("\\s+", " ", text)
    return text

In [3]:
# this is used to remove the word "SPEECH" adn the number following after that in the corpus
def remove_SPEECH(text):
    text = re.sub("SPEECH \d+", "", text)
    text = re.sub("\\s+", " ", text)
    return text

In [91]:
path = '/Users/nico_marais/code/Liedjieboer/forward_farming/NLP/Text_data/The-Jargon-File.txt'
text = open(path).read()
text_into_list = text.replace('\n', ' ').split(".")

In [93]:
text_into_list

['======= THIS IS THE JARGON FILE, VERSION 4',
 '2',
 '2, 20 AUG 2000 =======   This is the Jargon File, a comprehensive compendium of hacker slang  illuminating many aspects of hackish tradition, folklore, and humor',
 '  This document (the Jargon File) is in the public domain, to be freely  used, shared, and modified',
 ' There are (by intention) no legal  restraints on what you can do with it, but there are traditions about  its proper use to which many hackers are quite strongly attached',
 '  Please extend the courtesy of proper citation when you quote the File,  ideally with a version number, as it will change and grow over time',
 '  (Examples of appropriate citation form: "Jargon File 4',
 '2',
 '2" or "The  on-line hacker Jargon File, version 4',
 '2',
 '2, 20 AUG 2000"',
 ')  The Jargon File is a common heritage of the hacker culture',
 ' Over the  years a number of individuals have volunteered considerable time to  maintaining the File and been recognized by the net at large

In [94]:
# read all files in the folder (need to be txt with UTF-8 encoding)
# chop it up in sentances (for Tokenizer)
# for filename in os.listdir(path):
#     text = ''.join(open(path+'/'+filename, encoding = 'cp1252', mode="r").readlines())
split_text = re.split(r' *[\.\?!][\'"\)\]]* *', remove_stage_dir(text)) #change the function accordingly
for chunk in split_text:
        in_sentences.append(chunk)

print(text_into_list[0:10])
print('Corpus length:', len(text))

['======= THIS IS THE JARGON FILE, VERSION 4', '2', '2, 20 AUG 2000 =======   This is the Jargon File, a comprehensive compendium of hacker slang  illuminating many aspects of hackish tradition, folklore, and humor', '  This document (the Jargon File) is in the public domain, to be freely  used, shared, and modified', ' There are (by intention) no legal  restraints on what you can do with it, but there are traditions about  its proper use to which many hackers are quite strongly attached', '  Please extend the courtesy of proper citation when you quote the File,  ideally with a version number, as it will change and grow over time', '  (Examples of appropriate citation form: "Jargon File 4', '2', '2" or "The  on-line hacker Jargon File, version 4', '2']
Corpus length: 1680490


In [95]:
# Length of extracted sample
maxlen = 20

# Stride of sampling
step = 1

# This holds our samples sequences
sentences = []

# This holds the next word (as training label)
next_word = []

#use Kears Tokenizer
from keras.preprocessing.text import Tokenizer

max_num_word = 10000 #max size of library
tokenizer = Tokenizer(num_words=max_num_word)
tokenizer.fit_on_texts(list(in_sentences))
list_tokenized_train = tokenizer.texts_to_sequences(list(in_sentences))

#if the library ends up smaller then the max size, update the info
if len(tokenizer.word_index) < max_num_word:
    max_num_word = len(tokenizer.word_index)
    
print('Number of words:', max_num_word)

Number of words: 10000


In [96]:
#stick the encoded words back together as a big sequence
token_word = []
for line in range (0,len(in_sentences)):
    that_sentences = list_tokenized_train[line]
    for i in range(0,len(that_sentences)):
        token_word.append(that_sentences[i])

#sample the sequence
for i in range(0, len(token_word) - maxlen, step):
    sentences.append(token_word[i: i + maxlen])
    next_word.append(token_word[i + maxlen])
print('Number of sentences:', len(sentences))

Number of sentences: 1262803


In [97]:
#normalized x
x = np.asarray(sentences).astype('float32')/max_num_word
#one-hot encode y
y = np.zeros((len(sentences), max_num_word), dtype=np.bool)
for i in range (0,len(sentences)):
    for j in range (0,maxlen):
        y[i, next_word[j]] = 1

/var/folders/sq/vmyxb1t15h30lrlq40w06g2r0000gn/T/ipykernel_16215/3967043378.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), max_num_word), dtype=np.bool)


In [98]:
#build Keras model, using word embedding layer and LSTM then 
#output via softmax layer to give a prediction distribution
from keras import layers

model = Sequential()
model.add(layers.Embedding(max_num_word, 256, input_length=maxlen))
model.add(layers.LSTM(256))
model.add(layers.Dense(max_num_word, activation='softmax'))

model.summary()

# Since our prediction are one-hot encoded, use `categorical_crossentropy` as the loss
# optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.fit(x, y, batch_size=256, epochs=1)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 256)           2560000   
                                                                 
 lstm_5 (LSTM)               (None, 256)               525312    
                                                                 
 dense_5 (Dense)             (None, 10000)             2570000   
                                                                 
Total params: 5,655,312
Trainable params: 5,655,312
Non-trainable params: 0
_________________________________________________________________
4933/4933 [==============================] - 1082s 219ms/step - loss: 56.5712


In [99]:
#this is for sampling the next work with a prediction distribution
def sample(preds, temperature=0.1):
    preds = np.asarray(preds).astype('float64')
    exp_preds = preds - np.exp(temperature)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

#to change back to word
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

In [100]:
#randomize a seed
import random

start_index = random.randint(0, len(token_word) - maxlen - 1)
generated_seed = token_word[start_index: start_index + maxlen]

generated_text = ' '.join([reverse_word_map.get(i) for i in generated_seed])
print('--- Generating with seed ---')
print(generated_text)
print('--- --- --- --- --- ---')

for i in range(40): #generate 20 words

    array_seed = np.zeros((maxlen,1))
    array_seed[:,0] = np.asarray(generated_seed).astype('float32')/max_num_word
    
    preds = model.predict(array_seed.transpose(), verbose=0)[0]
    next_index = sample(preds)
    next_word = reverse_word_map.get(next_index)

    generated_seed.append(next_index)       
    generated_seed = generated_seed[1:]
    generated_text = generated_text + ' ' + next_word

print('--- Generated text ---')
print(generated_text)
print('--- --- --- --- --- ---')

--- Generating with seed ---
that she knew all about and their schemes and she did not propose to have a girl from them the
--- --- --- --- --- ---
--- Generated text ---
that she knew all about and their schemes and she did not propose to have a girl from them the art portmanteau pejorative trademark swap genii neck void intrinsic translate algorithm 297 drive fallen demands communicating east humility buzz we'll forming lady's managed batch foolishly suppressed pointed ideal et demons page ours newer temper —com folklore publication acts 533 359
--- --- --- --- --- ---
